AUTOMATED SUPPY CHAIN MANAGEMENT USING AGENTS- USECASE-4-CUSTOMER RECOMMENDATION-AWS HACK 2024-AGENT-4

Pre-requisites:

#This notebook requires permissions to:
- create and delete Amazon IAM roles
- access Amazon Bedrock

#The following access are required for Sagemaker to carry on the usecase:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess
- AmazonDynamoDBFullAccess

EDA and extraction of required columns

In [18]:
import pandas as pd


csv_file = r"Supply chain dataset - Supply chain dataset (2).csv" #loading csv
data = pd.read_csv(csv_file)


def get_customer_summary(customer_name):
    customer_data = data[data['CustomerName'] == customer_name]
    if customer_data.empty:
        return f"No data found for customer name: {customer_name}"    
    customer_info = customer_data[['CustomerID', 'CustomerEmail', 'PurchaseHistory']]


    summaries = []
    for _, row in customer_info.iterrows():
        summary = f"CustomerID {row['CustomerID']} with email {row['CustomerEmail']} has purchase history: {row['PurchaseHistory']}."
        summaries.append(summary)
    result = " ".join(summaries)
    return "customer name: " + customer_name + " " + result

customer_name = "Aarohi" 
summary = get_customer_summary(customer_name)

print(summary)


customer name: Aarohi CustomerID 1875 with email aarohi@example.com has purchase history: Smartphone. CustomerID 1875 with email aarohi@example.com has purchase history: Camera. CustomerID 1875 with email aarohi@example.com has purchase history: Monitor.


CREATING AGENT

In [3]:
!pip install boto3
!pip install opensearch-py
!pip install botocore
!pip install awscli
!pip install retrying

In [13]:
import boto3
import json
import time
import zipfile
import logging
import pprint
from io import BytesIO

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
dynamodb_client = boto3.client('dynamodb')
dynamodb_resource = boto3.resource('dynamodb')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)





def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)


def create_agent_role(agent_name, agent_foundation_model, kb_id=None):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    # Create IAM policies for agent
    statements = [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
    # add Knowledge Base retrieve and retrieve and generate permissions if agent has KB attached to it
    if kb_id:
        statements.append(
            {
                "Sid": "QueryKB",
                "Effect": "Allow",
                "Action": [
                    "bedrock:Retrieve",
                    "bedrock:RetrieveAndGenerate"
                ],
                "Resource": [
                    f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
                ]
            }
        )

    bedrock_agent_bedrock_allow_policy_statement = {
        "Version": "2012-10-17",
        "Statement": statements
    }

    bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
    try:
        agent_bedrock_policy = iam_client.create_policy(
            PolicyName=agent_bedrock_allow_policy_name,
            PolicyDocument=bedrock_policy_json
        )
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_bedrock_policy = iam_client.get_policy(
            PolicyArn=f"arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}"
        )
                    
    # Create IAM Role for the agent and attach IAM policies
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)
    try:
        agent_role = iam_client.create_role(
            RoleName=agent_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json
        )

        # Pause to make sure role is created
        time.sleep(10)
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_role = iam_client.get_role(
            RoleName=agent_role_name,
        )

    iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
    return agent_role


def delete_agent_roles_and_policies(agent_name, kb_policy_name):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    dynamodb_access_policy_name = f'{agent_name}-dynamodb-policy'
    lambda_function_role = f'{agent_name}-lambda-role'

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=agent_role_name,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {agent_role_name}")
            print(e)

    for policy in [dynamodb_access_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=lambda_function_role,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {lambda_function_role}")
            print(e)

    try:
        iam_client.detach_role_policy(
            RoleName=lambda_function_role,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
    except Exception as e:
        print(f"Could not detach AWSLambdaBasicExecutionRole from {lambda_function_role}")
        print(e)

    for role_name in [agent_role_name, lambda_function_role]:
        try:
            iam_client.delete_role(
                RoleName=role_name
            )
        except Exception as e:
            print(f"Could not delete role {role_name}")
            print(e)

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name, dynamodb_access_policy_name]:
        try:
            iam_client.delete_policy(
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not delete policy {policy}")
            print(e)


def clean_up_resources(
        table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions,
        agent_id, kb_id, alias_id
):
    action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
    action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']
    # Delete Agent Action Group, Agent Alias, and Agent
    try:
        bedrock_agent_client.update_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId= action_group_id,
            actionGroupName=action_group_name,
            actionGroupExecutor={
                'lambda': lambda_function['FunctionArn']
            },
            functionSchema={
                'functions': agent_functions
            },
            actionGroupState='DISABLED',
        )
        bedrock_agent_client.disassociate_agent_knowledge_base(
            agentId=agent_id,
            agentVersion='DRAFT',
            knowledgeBaseId=kb_id
        )
        bedrock_agent_client.delete_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId=action_group_id
        )
        bedrock_agent_client.delete_agent_alias(
            agentAliasId=alias_id,
            agentId=agent_id
        )
        bedrock_agent_client.delete_agent(agentId=agent_id)
        print(f"Agent {agent_id}, Agent Alias {alias_id}, and Action Group have been deleted.")
    except Exception as e:
        print(f"Error deleting Agent resources: {e}")

    # Delete Lambda function
    try:
        lambda_client.delete_function(FunctionName=lambda_function_name)
        print(f"Lambda function {lambda_function_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting Lambda function {lambda_function_name}: {e}")

    # Delete DynamoDB table
    try:
        dynamodb_client.delete_table(TableName=table_name)
        print(f"Table {table_name} is being deleted...")
        waiter = dynamodb_client.get_waiter('table_not_exists')
        waiter.wait(TableName=table_name)
        print(f"Table {table_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting table {table_name}: {e}")

In [14]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid




In [15]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-east-1', '199328835594')

In [17]:
suffix = f"{region}-{account_id}"
agent_name = 'supply-chain-agent-awshack'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of querying and giving valueable insights and makes customer recommendations"
agent_instruction = """
You are a supply chain management agent. You look into the current data and give good  recommenations on what the customers might buy based on their previous spends
"""

agent_action_group_description = """
Creating recommendations for customers"""

agent_action_group_name = "Recommendationcreator"

In [29]:
agent_foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
agent_foundation_model

'anthropic.claude-3-sonnet-20240229-v1:0'

In [30]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

In [31]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_supply-chain-agent-awshack',
  'RoleId': 'AROAS42HMVAFLBQDTEZGH',
  'Arn': 'arn:aws:iam::199328835594:role/AmazonBedrockExecutionRoleForAgents_supply-chain-agent-awshack',
  'CreateDate': datetime.datetime(2024, 8, 25, 13, 40, 25, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]},
  'MaxSessionDuration': 3600,
  'RoleLastUsed': {'LastUsedDate': datetime.datetime(2024, 8, 25, 13, 47, 36, tzinfo=tzlocal()),
   'Region': 'us-east-1'}},
 'ResponseMetadata': {'RequestId': '1cca9924-e65e-433b-b291-761b9d6d6742',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 07:01:42 GMT',
   'x-amzn-requestid': '1cca9924-e65e-433b-b291-761b9d6d6742',
   'content-type': 'text/xml',
   'content-length': '1028'},
  'RetryAttempts': 0}}

In [32]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

ConflictException: An error occurred (ConflictException) when calling the CreateAgent operation: Could not perform Create operation, since the supply-chain-agent-awshack (id: B50FK1NSDE) with the same name supply-chain-agent-awshack already exists

In [33]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

NameError: name 'response' is not defined

In [35]:
agent_id = 'B50FK1NSDE'

In [37]:
agent_functions = [
    
    {
        'name': 'create_recommendation',
        'description': 'create a recommendation for the customer',
        'parameters': {
            "CustomerName": {
                "description": "The name of the customer",
                "required": True,
                "type": "string"
            },
            "CustomerEmail": {
                "description": "Email id of the customer",
                "required": True,
                "type": "string"
            },
            "recommendation": {
                "description": "Product recommended to the customer",
                "required": True,
                "type": "string"
            }
        }
    },
    
]

In [38]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda':'arn:aws:lambda:us-east-1:199328835594:function:bedrock_agent_lambda'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

ConflictException: An error occurred (ConflictException) when calling the CreateAgentActionGroup operation: Could not perform Create operation, since the Recommendationcreator (id: L99TSYX2LT) with the same name Recommendationcreator already exists

In [ ]:
agent_action_group_response

In [ ]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName= 'bedrock_agent_lambda',
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

In [39]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': '014b70d9-93ed-40a1-9711-9f995099e324', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 07:05:26 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '014b70d9-93ed-40a1-9711-9f995099e324', 'x-amz-apigw-id': 'dGtgfGjYIAMEalw=', 'x-amzn-trace-id': 'Root=1-66cc2936-64a9bee91ca3b2677df7a09c'}, 'RetryAttempts': 0}, 'agentId': 'B50FK1NSDE', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 8, 26, 7, 5, 26, 159212, tzinfo=tzlocal())}


In [19]:
alias_id = 'TSTALIASID'
agent_id = 'B50FK1NSDE'

In [20]:
%%time
session_id:str = str(uuid.uuid1())
query = summary + "Print all this info and recommend only 5 prodcuts based on the customers purchase history."

response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Dear Aarohi,

Based on your purchase history of a smartphone, camera, and monitor, here are some product recommendations that may interest you:

1. External Storage: Since you have a smartphone and camera, an external hard drive or cloud storage subscription could help you backup and organize your photos and videos.

2. Laptop or Desktop Computer: With a monitor already purchased, a new laptop or desktop computer could complement your setup for productivity, entertainment, or creative tasks.

3. Photography Accessories: Considering your interest in cameras, you may find tripods, lenses, camera bags, or photo editing software useful for enhancing your photography experience.

4. Smart Home Devices: To integrate with your existing electronics, you could explore smart home products like smart speakers, smart displays, or home automation systems.

5. Portable Power Bank: A power bank or portable charger can be handy for keeping your smartphone and other devices charged on the go.

These re